In [ ]:
import sys
sys.path.append("../src")
import pyvista as pv
import numpy as np
import nibabel as nib
import kimimaro
from scipy.spatial import Delaunay

import matplotlib.pyplot as plt

from utils.plotting import plot_skeleton_3d

file = "../data/nnUNet_raw/Dataset042_small_bowel/labelsTs/s0006.nii.gz"
file = nib.load(file)
ground_truth = np.asarray(file.dataobj)
print(np.unique(ground_truth))

skels = kimimaro.skeletonize(
    ground_truth,
    teasar_params={
        "scale": 3,
        "const": 5,
        "pdrf_scale": 10000,
        "pdrf_exponent": 4,
        "soma_acceptance_threshold": 3500,  # physical units
        # "soma_detection_threshold": 750,  # physical units
        # "soma_invalidation_const": 300,  # physical units
        # "soma_invalidation_scale": 2,
    },
    anisotropy=(1, 1, 1),
    dust_threshold=5,
    fix_branching=True,
    progress=True,
    parallel_chunk_size=100, # for the progress bar
)

skeleton = skels[1]
print(skeleton.cable_length())
# skeleton.viewer()

# Idea: Discretize the voxel space in coordinates

In [ ]:
delaunay = Delaunay(np.asarray(ground_truth.nonzero()).T)
print(len(delaunay.simplices))

np.concatenate([np.full(delaunay.simplices.shape[0],4).reshape(-1,1), delaunay.simplices], axis=1)

In [ ]:
skely_delauney = Delaunay(skeleton.vertices)

skeleton_3d = pv.PolyData(skely_delauney.points, strips=np.concatenate((np.full(skely_delauney.simplices.shape[0],4).reshape(-1,1), skely_delauney.simplices), axis=1).flatten())

skeleton_3d.plot(opacity=0.3, show_edges=True)

In [ ]:
(skely_delauney.simplices).shape
# Low-energy model that can find the parameters of the twist and turns, the hyperparameters of the tube, and maybe some neural network that can predict the tube from the volume
# Generate a Voronoi-like diagram which might find a boundary between the tube and the volume
# Superpixel/super-voxel-based segmentation
# If we are going into RL, we need to have a reward function that can be examined at each step of the optimization; Martin expresses the opinion that RL should be the last step for refining the model.  

In [ ]:
# additional_points = skeleton.vertices[skely_delauney.simplices].mean(axis=1).astype(int)
# print(len(additional_points))

# _additional_points = {}
# _ignore = {}
# for i, point in enumerate(additional_points):
#     tup = tuple(point)
#     if tup in _additional_points:
#         _ignore[i] = i
#     else:
#         _additional_points[i]=tup
# additional_points = np.array(list(_additional_points.values()))

# skeleton_2 = skeleton.clone()
# skeleton_2.vertices = np.concatenate((skeleton.vertices, additional_points), axis=0)

# skely_delauney.simplices 

# additional_points_idx = np.arange(len(skeleton.vertices), len(skeleton.vertices) + len(additional_points)).reshape(-1,1)

# print(skeleton_2.edges.shape)
# additional_edges = []
# simplices = skely_delauney.simplices[~np.isin(np.arange(len(skely_delauney.simplices)), _ignore)]
# for i in range(simplices.shape[1]):
#     additional_edges.append(np.concatenate((simplices[:, i].reshape(-1,1), additional_points_idx), axis=1))
# additional_edges = np.concatenate(additional_edges, axis=0)

# skeleton_2.edges = np.concatenate((skeleton.edges, additional_edges + len(skeleton.vertices)), axis=0)
# skeleton_2.radii = np.concatenate((skeleton.radii, np.full(len(additional_edges), 1)), axis=0)
# print(len(skeleton_2.edges))

plotter = pv.Plotter()
# plotter.add_mesh(skeleton_2.vertices, color='red')
# plotter.add_mesh(skeleton.vertices, color='blue')
# plotter.show()


def plot_skeleton_3d(skeleton, volume = None):
    """Plot a 3D skeleton with pyvista

    Parameters
    ----------
    skeleton : cloudvolume.Skeleton
        The skeleton to plot. Should contain the vertices, edges and radii
    volume : np.ndarray, optional
        The volume segmentation mask to plot. Default is None.
    """
    plotter = pv.Plotter()

    skeleton_3d = pv.PolyData(
        skeleton.vertices,
        lines=np.concatenate((np.full(skeleton.edges.shape[0], 2).reshape(-1, 1), skeleton.edges), axis=1),
    )

    if hasattr(skeleton, "radii"):
        # Not sure why we need to remove one element, but for some reason the polydata is one element less.
        skeleton_3d.cell_data["width"] = skeleton.radii[1:]

    plotter.add_mesh(skeleton_3d, show_edges=True, line_width=5, scalars="width")

    if volume is not None:
        plotter.add_volume(volume * 20, cmap="viridis", specular=0.5, specular_power=15)

    plotter.view_xz()
    plotter.show()

plot_skeleton_3d(skeleton, ground_truth)

In [ ]:
skels[1]??
# Algorithm for post-processing of the skeleton
# 1. Add weak edges to the outside points of the skeleton (using Delaunay points)
# 1.5. Add weak edges with a neighbourhood of ~3. 
# 2. Split the nodes with radius > T into two nodes (iteratively, until all nodes have radius < T)
# 3. Add connections for the split nodes (from beginning to end, e.g. 1-2-3 -> 1-2, 2-3, 1-4, 4-3)

In [ ]:
import vtk
import pyvista as pv
import nibabel as nib
import numpy as np

def nii_2_mesh(filename_nii, filename_stl, label):
    """
    Read a nifti file including a binary map of a segmented organ with label id = label.
    Convert it to a smoothed mesh of type stl.

    filename_nii     : Input nifti binary map
    filename_stl     : Output mesh name in stl format
    label            : segmented label id
    
    Courtesy of: https://github.com/MahsaShk/MeshProcessing/blob/master/nii_2_mesh_conversion.py
    """

    # read the file
    reader = vtk.vtkNIFTIImageReader()
    reader.SetFileName(filename_nii)
    reader.Update()

    # apply marching cube surface generation
    surf = vtk.vtkDiscreteMarchingCubes()
    surf.SetInputConnection(reader.GetOutputPort())
    surf.SetValue(0, label)  # use surf.GenerateValues function if more than one contour is available in the file
    surf.Update()

    # smoothing the mesh
    smoother = vtk.vtkWindowedSincPolyDataFilter()
    if vtk.VTK_MAJOR_VERSION <= 5:
        smoother.SetInput(surf.GetOutput())
    else:
        smoother.SetInputConnection(surf.GetOutputPort())
    smoother.SetNumberOfIterations(30)
    smoother.NonManifoldSmoothingOn()
    smoother.NormalizeCoordinatesOn()  # The positions can be translated and scaled such that they fit within a range of [-1, 1] prior to the smoothing computation
    smoother.GenerateErrorScalarsOn()
    smoother.Update()

    # save the output
    writer = vtk.vtkSTLWriter()
    writer.SetInputConnection(smoother.GetOutputPort())
    writer.SetFileTypeToASCII()
    writer.SetFileName(filename_stl)
    writer.Write()


filename_ct = "../data/nnUNet_raw/Dataset042_small_bowel/imagesTr/s0006_0000.nii.gz"
filename_gt = "../data/nnUNet_raw/Dataset042_small_bowel/labelsTr/s0006.nii.gz"
filename_stl = "01.stl"
# nii_2_mesh(filename_nii, filename_stl, 1)
ground_truth = np.asarray(nib.load(filename_gt).dataobj)

def normalize_ct(nii, percentiles: tuple[float, float] = (0.0005, 0.9995), window: tuple[float, float] =(50, 400)):
    nii = np.where((nii > np.quantile(nii, percentiles[0])) & (nii < (np.quantile(nii, percentiles[1]))), nii, 0)
    window_c, window_w = window
    nii = np.where((nii >= window_c - window_w/2) & (nii <= window_c + window_w/2), nii, 0)
    # nii = nii / np.linalg.norm(nii, axis=-1, keepdims=True)
    return nii

def load_and_normalize(filename, percentiles=(0.0005, 0.9995), window=(50, 400)):
    nii = nib.load(filename)
    nii = np.asarray(nii.dataobj)
    nii = normalize_ct(nii, percentiles, window)
    return nii

def save_nifti(data, filename, other = None):
    if other is None:
        other = nib.load(filename)
    else:
        other = nib.load(other)
    new_image = nib.Nifti1Image(data, other.affine, other.header)
    nib.save(new_image, filename)


: 

# Discussion (05/02/2025)
What you want to model is the flow of two cells: and the flow of the two cells is defined by the surface of the cells. The flow goes orthogonal to the surface. Therefore you should go into the direction of the path and not orthogonal to the path.
We want to quantify the cost of crossing a wall, not just the cost of all pixels even the ones which don't relate.
